In [ ]:
!pip install ibm-watsonx-ai

In [ ]:
!pip install pypdf

In [ ]:
pip install PyPDF2

In [3]:
from credentials import *
import os

In [4]:
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference

credentials = Credentials(
    url=ENDPOINT_URL,
    api_key=API_KEY
)


In [8]:
import PyPDF2

Model

In [54]:
model = ModelInference(
    model_id="meta-llama/llama-3-405b-instruct", # Che conosciamo bene 😊🏆 
    credentials=credentials,
    project_id=PROJECT_ID,
    params={
            "max_tokens": 20000
        }
)

In [ ]:
def validate_json_output(result):
    try:
        # Rimuovi eventuali backtick e indicatori di codice
        json_str = result.replace('```json', '').replace('```', '').strip()
        # Parse del JSON
        json_obj = json.loads(json_str)
        
        # Verifica struttura base
        required_fields = ['restaurant']
        for field in required_fields:
            if field not in json_obj:
                print(f"Campo mancante: {field}")
                return False
                
        # Se la validazione passa, restituisci il JSON formattato
        return json.dumps(json_obj, indent=2)
    except Exception as e:
        print(f"Errore nella validazione JSON: {str(e)}")
        return False

def validate_and_store_result(result):
    validated_json = validate_json_output(result)
    if validated_json:
        return validated_json
    else:
        return None

## Parte 2

STRUTTURA JSON 

In [49]:
json_structure = """
{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "Ristorante L'Eco di Pandora",
  "type": "object",
  "properties": {
    "nome": {
      "type": "string",
      "description": "Nome del ristorante"
    },
    "chef": {
      "type": "string",
      "description": "Nome del chef"
    },
    "descrizione": {
      "type": "string",
      "description": "Descrizione del ristorante"
    },
    "licenze": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "skill": {
            "type": "string",
            "description": "Nome della licenza"
          },
          "livello": {
            "type": "integer",
            "description": "Livello della licenza"
          }
        },
        "required": ["skill", "livello"]
      },
      "description": "Elenco delle licenze del ristorante"
    },
    "menu": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "nome": {
            "type": "string",
            "description": "Nome del piatto"
          },
          "descrizione": {
            "type": "string",
            "description": "Descrizione del piatto"
          },
          "ingredienti": {
            "type": "array",
            "items": {
              "type": "string",
              "description": "Ingredienti del piatto"
            },
            "description": "Elenco degli ingredienti del piatto"
          },
          "tecniche": {
            "type": "array",
            "items": {
              "type": "string",
              "description": "Tecniche utilizzate per preparare il piatto"
            },
            "description": "Elenco delle tecniche utilizzate per preparare il piatto"
          }
        },
        "required": ["nome", "descrizione", "ingredienti", "tecniche"]
      "description": "Elenco dei piatti del menu"
    }
  },
  "required": ["nome", "chef", "descrizione", "licenze", "menu"]
}
"""

In [50]:
main_prompt = f"""
Sei un esperto analizzatore di documenti in ambito ristorazione galattica e fantasy. 
Il tuo compito è analizzare documenti di menù e estrarre informazioni strutturate secondo uno schema JSON predefinito.

REGOLE IMPORTANTI:
- Devi mantenere la stessa struttura JSON fornita
- Compila TUTTI i campi, non lasciare valori nulli
- Se un'informazione non è presente nel testo, usa valori plausibili basati sul contesto
- Mantieni la coerenza tra le varie parti del documento

CONTESTO SPECIFICO:
- I documenti provengono da ristoranti in un universo fantasy/sci-fi
- Ogni chef ha abilità speciali e licenze specifiche
- Le tecniche di cucina possono includere elementi magici o tecnologici
- Gli ingredienti possono provenire da diverse dimensioni o pianeti

REGOLA DA SEGUIRE:
RISPONDI SOLO CON IL JSON, NON AGGIUNGGERE COMMENTI O INTESTAZIONI

STRUTTURA DA POPOLARE:
{json_structure}

DOCUMENTO DA ANALIZZARE:
"""

FUNZIONI DI PROCESSAMENTO

LOOP PRINCIPALE

Install Mongo

In [ ]:
!pip install pymongo

In [45]:
def pdf_to_text(doc_path, index):
    try:
        with open(doc_path, "rb") as f:
            pdf = PyPDF2.PdfReader(f)
            text = f"\nDocumento {index}:\n"
            for page in pdf.pages:
                text += page.extract_text()
            return text
    except Exception as e:
        print(f"Errore nel processare {doc_path}: {e}")
        return None

for index, doc in enumerate(os.listdir("Hackapizza Dataset/Menu/")[0:1]):
    doc_path = f"Hackapizza Dataset/Menu/{doc}"
    processed_doc = pdf_to_text(doc_path, index)
    print(processed_doc)


Documento 0:
Ristorante "Anima Cosmica"
Chef Aurora Stellaris
Nel cuore pulsante di Pandora, dove le foreste bioluminescenti e le montagne fluttuanti si uniscono in un
mosaico vibrante, emerge un gioiello culinario che fonde la magia di questo mondo alieno con l'eccellenza
gastronomica. L'Anima Cosmica, guidato dall'innovativa Chef Aurora Stellaris, è il vertice della cucina
pandoriana, una sintesi di tradizione Na'vi e avanzata scienza culinaria.
Il viaggio della Chef Stellaris verso la maestria culinaria iniziò in seguito a un evento trasformativo che
risvegliò in lei straordinarie capacità psioniche. Queste abilità le permettono di percepire l'interconnessione
tra la natura rigogliosa di Pandora e gli ingredienti che utilizza. In questo luogo incantato, i suoi strumenti di
cucina diventano estensioni del suo essere, manovrati con precisione telecinetica, preparando piatti che
riflettono le molteplici dimensioni di questo pianeta meraviglioso.
La sua conoscenza delle forze gravitazi

In [58]:
from pymongo import MongoClient
import json
import os
import PyPDF2

# Connessione MongoDB
try:
    client = MongoClient('mongodb://localhost:27017/')
    db = client['hackapizza']
    collection = db['restaurants']
    print("Connessione a MongoDB riuscita!")
    # Pulisci la collection all'inizio
    #collection.drop()
except Exception as e:
    print(f"Errore di connessione MongoDB: {e}")

def pdf_to_text(doc_path, index):
    try:
        with open(doc_path, "rb") as f:
            pdf = PyPDF2.PdfReader(f)
            text = f"\nDocumento {index}:\n"
            for page in pdf.pages:
                text += page.extract_text()
            return text
    except Exception as e:
        print(f"Errore nel processare {doc_path}: {e}")
        return None


def process_document(prompt, model):
    try:
        result = model.chat(messages=[{'role': 'user', 'content': prompt}])
        return result['choices'][0]['message']['content']
    except Exception as e:
        print(f"Errore nell'elaborazione: {e}")
        return None

def clean_and_parse_json(json_string):
    try:
        # Rimuovi eventuali caratteri di markdown
        clean_string = json_string.replace('```json', '').replace('```', '').strip()
        # Converti in dizionario Python
        return json.loads(clean_string)
    except json.JSONDecodeError as e:
        print(f"Errore nel parsing JSON: {e}")
        return None

# Inizializzazione
batch_size = 1
documents = []
last_output = ""

# Loop principale
for index, doc in enumerate(os.listdir("Hackapizza Dataset/Menu/")[2:3]):
    doc_path = f"Hackapizza Dataset/Menu/{doc}"
    processed_doc = pdf_to_text(doc_path, index)
    
    # if processed_doc:
    #     documents.append(processed_doc)
    
    # if len(documents) == batch_size or index == len(os.listdir("Hackapizza Dataset/Menu")) - 1:
        #prompt = create_batch_prompt(documents, main_prompt, last_output)
    prompt = main_prompt + processed_doc
    result = process_document(prompt, model)
    print(result)


    if result:
        try:
            # Converti il risultato in JSON
            json_data = clean_and_parse_json(result)
            if json_data:
                # Inserisci in MongoDB
                collection.insert_one(json_data)
                print(f"Documento inserito con successo in MongoDB")
                last_output = result
        except Exception as e:
            print(f"Errore nell'inserimento in MongoDB: {e}")
    
    documents = []

# Verifica finale
print(f"\nTotale documenti inseriti: {collection.count_documents({})}")
print("\nRistoranti inseriti:")
for doc in collection.find():
    if 'nome' in doc:
        print(f"- {doc['nome']}")
    else:
        print("- [Nome non trovato]")

Connessione a MongoDB riuscita!
```
{
  "nome": "Cosmica Essenza",
  "chef": "Aurelio Celestini",
  "descrizione": "Ristorante di alta cucina galattica sulla pianeta Tatooine",
  "licenze": [
    {
      "skill": "Psionica",
      "livello": 5
    },
    {
      "skill": "Quantica",
      "livello": 9
    },
    {
      "skill": "Controllo Gravitazionale",
      "livello": 3
    },
    {
      "skill": "Antimateria",
      "livello": 1
    },
    {
      "skill": "Luce",
      "livello": 2
    },
    {
      "skill": "LTK",
      "livello": 3
    }
  ],
  "menu": [
    {
      "nome": "Galassie in Epifania",
      "descrizione": "Risotto Celestiale con Preziosi dell'Universo",
      "ingredienti": [
        "Polvere di Pulsar",
        "Carne di Xenodonte",
        "Carne di Balena spaziale",
        "Teste di Idra",
        "Spaghi del Sole",
        "Riso di Cassandra",
        "Farina di Nettuno",
        "Lacrime di Andromeda",
        "Petali di Eco"
      ],
      "tecniche": [
 